In [ ]:
# FinanceDataReader를 사용하여 KRX 주식 코드 목록을 가져옵니다
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

# 개별 종목의 종가와 거래량 데이터를 가져와 데이터 프레임을 만듭니다
df_individual_stocks = pd.DataFrame()

# 첫 번째 컬럼의 이름인 Code를 반복해서 넣어주는 반복문을 만듭니다
for code in df_krx['Code']:
    # 각 종목의 종가와 거래량 데이터를 가져옵니다
    stock_data = fdr.DataReader(code, '2023-01-01', '2023-12-31')
    
    # 종가와 거래량 데이터를 데이터프레임에 추가합니다
    df_individual_stocks[code + '_Close'] = stock_data['Close']
    df_individual_stocks[code + '_Volume'] = stock_data['Volume']

# 가지고온 종가와 거래량 데이터를 데이터 프레임에 추가합니다
# 인덱스를 시계열 데이터로 변경합니다
df_individual_stocks.index = pd.to_datetime(stock_data.index)

In [ ]:
# FinanceDataReader를 사용하여 KRX 주식 코드 목록을 가져옵니다
import FinanceDataReader as fdr
df_krx = fdr.StockListing('KRX')

# df_krx DataFrame의 첫 번째 열(종목코드)을 사용하여 주식 코드 목록을 얻습니다
stock_codes = df_krx.iloc[:, 0].tolist()

# 빈 DataFrame을 생성합니다
df = pd.DataFrame()

# 각 주식 코드에 대해 2023년의 종가와 거래량 데이터를 가져와서 DataFrame에 추가합니다
for code in stock_codes:
    # 각 주식 코드에 대해 2023년의 종가와 거래량 데이터를 가져옵니다
    stock_data_2023 = fdr.DataReader(code, '2023-01-01', '2023-12-31')
    
    # 종가와 거래량 데이터를 DataFrame에 추가합니다
    df[code + '_Close'] = stock_data_2023['Close']
    df[code + '_Volume'] = stock_data_2023['Volume']

# 인덱스를 시계열 데이터로 변환합니다
df.index = pd.to_datetime(df.index)

In [ ]:
df_krx

In [ ]:
from pandas_datareader import data as pdr

import yfinance as yf
yf.pdr_override()

import numpy as np
import pandas as pd

import FinanceDataReader as fdr

df = fdr.DataReader('005930','2018')

df.head()

In [ ]:
# 종가만 가져올 수 있겠죠!

df[['Close']]

In [ ]:
# 변동이 20% 이상인 날들만 가져와보기

df[abs(df['Change']) > 0.05]

In [ ]:
# 주가 그래프 그려보기

df = fdr.DataReader('005930','2018')
df.plot(y=['Open','Close'],figsize=(15,8),grid=True)

In [ ]:
# 두 종목의 주가 변동 그래프 그려보기

df_1 = fdr.DataReader('005930','2018')
df_2 = fdr.DataReader('066570','2018')

df_tot = pd.DataFrame()
df_tot['Samsung'] = df_1[['Change']]
df_tot['LG'] = df_2[['Change']]

df_tot.tail(100).plot(figsize=(15,8))

In [ ]:
# 이동평균선을 만들고 매수, 매도 시점 확인

def get_return(code,n):
  df = fdr.DataReader(code,'2018')

  df = df[['Close']].copy()

  df['ma'] = df.rolling(n).mean().shift(1)

  df['action'] = np.where(df['Close'] > df['ma'], 'buy', 'sell')

  df.iloc[-1,-1] = 'sell'

  cond1 = (df['action'] == 'buy') & (df['action'].shift(1) == 'sell')
  cond2 = (df['action'] == 'sell') & (df['action'].shift(1) == 'buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜','종가(buy)','이평값','액션']

  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜','종가(sell)','이평값','액션']

  df_result = pd.concat([df_buy,df_sell],axis=1)

  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']

  return df_result[['수익률']].cumprod().iloc[-1,-1] - 1

In [ ]:
get_return('005930',3)

In [ ]:
# 단기, 장기 이동평균선을 만들고 매수, 매도 시점 확인

def get_return_sl(code, short, long):
  df = fdr.DataReader(code,'2023')

  df = df[['Close']].copy()

  df['ma1'] = df['Close'].rolling(short).mean().shift(1)
  df['ma2'] = df['Close'].rolling(long).mean().shift(1)

  df['action'] = np.where(df['ma1'] > df['ma2'], 'buy', 'sell')

  df.iloc[-1,-1] = 'sell'

  cond1 = (df['action'] == 'buy') & (df['action'].shift(1) == 'sell')
  cond2 = (df['action'] == 'sell') & (df['action'].shift(1) == 'buy')

  df_buy = df[cond1].reset_index()
  df_buy.columns = ['날짜','종가(buy)','이평값1','이평값2','액션']

  df_sell = df[cond2].reset_index()
  df_sell.columns = ['날짜','종가(sell)','이평값1','이평값2','액션']

  df_result = pd.concat([df_buy,df_sell],axis=1)

  df_result['수익률'] = df_result['종가(sell)'] / df_result['종가(buy)']

  df_final = (df_result[['수익률']].cumprod().tail(1) - 1)*100
  df_final['단기'] = short
  df_final['장기'] = long

  return df_final

In [ ]:
get_return_sl('005930',3,30)

In [ ]:
# 최적의 단기, 장기 이동평균선 찾기

dfs = []
for short in range(3,11):
  for long in range(30,61):
    df = get_return_sl('005930',short,long)
    dfs.append(df)

df_result = pd.concat(dfs)
df_result.sort_values(by='수익률', ascending=False)